# Performing baselie subtraction

The baseline-subtracted histograms of an acquisition with 2 active channels were obtained. The baseline value for each channel is the mean of the readings from all the events triggered ONLY by external HOLD assertion. This value is subtracted from the values of each "real" event (triggered from internal HOLD).

The dataset for this analysis is a background acquisition of 10 000 events. SIPHRA channels 2 and 14 were connected to SiPM channels 1-4 and 5-8, respectively, using charge comparator, with a threshold value of 20.

In [1]:
import ROOT
import pandas as pd
import numpy as np
import os
import sys
import ipynbname
from pathlib import Path

project_root = str(ipynbname.path().parent.parent)
sys.path.append(project_root)

from processing import SiphraAcquisition, fit_peak_expbg

# ROOT.gROOT.SetBatch(False)
# ROOT.gEnv.SetValue("Canvas.ShowEventStatus", 1)
# ROOT.gEnv.SetValue("Canvas.ShowToolBar", 1)
# ROOT.gEnv.SetValue("Canvas.UseGL", 0)
# ROOT.gROOT.ProcessLine("gVirtualX = new TGX11();")


In [2]:
# Constants
BITS12 = 2**12
BITS9 = 2**9 # 512 typical number of bins used
# Energy emission peaks in MeV
K40_MEV = 1.460
TL208_MEV = 2.614

In [3]:
# Datasets
acq_notSubtracted = SiphraAcquisition(project_root+'/data/260203/1_SiPM_ChannelsTest_Ch1-4_Ch2_Ch5-8_Ch14_QT_Thr20_Hys0_Background.csv',
                                   active_chs=[2,14],
                                   exposure_sec=1)
acq_subtracted = SiphraAcquisition(project_root+'/data/260203/1_datdecodertest_subtractBaseline.csv',
                                      active_chs=[2,14],
                                      exposure_sec=1)

colors = [ROOT.kRed, ROOT.kBlue, ROOT.kGreen, ROOT.kOrange, ROOT.kViolet, ROOT.kYellow, ROOT.kSpring, ROOT.kCyan,]

In [61]:
if ROOT.gROOT.FindObject('cv'):
    ROOT.gROOT.FindObject('cv').Close()

SUMMED_XR=5000 # Right limit of the summed channel's x-axis

c = ROOT.TCanvas('cv', 'cv', 1200, 800)
ROOT.gStyle.SetOptStat(11)
ROOT.gStyle.SetStatFontSize(0.03)
ROOT.gStyle.SetStatW(0.16)

names=['BL subt.', 'No correction']
hists = []

lg = ROOT.TLegend(0.48, 0.61, 0.75, 0.83)

Yinit = 0.82 # For stat boxes

for idx, (acq, name) in enumerate(zip([acq_subtracted, acq_notSubtracted], names)):
    # print(f"Current file: {acq.filepath.name}")
    ch = 'Summed' #acq.active_chs[0]
    hist = ROOT.TH1F(f"{ch} {name}", "", int(BITS9*SUMMED_XR/BITS12), 0, SUMMED_XR)
    hist_singlech = ROOT.TH1F(f"Ch. {acq.active_chs[0]} {name}", "", BITS9, 0, BITS12)
    hist.Fill(acq[ch]/len(acq.active_chs))
    hist_singlech.Fill(acq[acq.active_chs[0]])
    hist.Scale(1/acq.exposure)
    hist_singlech.Scale(1/acq.exposure)

    hist_singlech2 = ROOT.TH1F(f"Ch. {acq.active_chs[1]} {name}", "", BITS9, 0, BITS12)
    hist_singlech2.Fill(acq[acq.active_chs[1]])
    hist_singlech2.Scale(1/acq.exposure)
    #Preeting up..
    hist.GetXaxis().SetTitle("ADC channel number")
    hist.GetYaxis().SetTitle(r"Counts")
    hist.SetLineColor(colors[idx])
    hist.SetTitle("Baseline subtraction comparison")
    # hist.GetYaxis().SetRangeUser(0, 1e4)
    lg.SetHeader("SIPHRA Channel")
    lg.AddEntry(hist, f"{ch} {name}", 'l')
    hists.append(hist)
    hists[-1].Draw('sames hist')
    # hist_singlech.GetXaxis().SetTitle("ADC channel number")
    # hist_singlech.GetYaxis().SetTitle(r"Count rate (s^{-1})")
    hist_singlech.SetLineColor(colors[idx + 2]+2)
    hist_singlech2.SetLineColor(colors[idx + 4]+1)
    # hist_singlech.SetTitle("Baseline subtraction comparison")
    lg.AddEntry(hist_singlech, f"Ch. {acq.active_chs[0]} {name}", 'l')
    lg.AddEntry(hist_singlech2, f"Ch. {acq.active_chs[1]} {name}", 'l')
    hists.append(hist_singlech)
    hists[-1].Draw('sames hist')
    hists.append(hist_singlech2)
    hists[-1].Draw('sames hist')
c.SetLogy()
ROOT.gPad.Update()
for i, sp in enumerate(hists):
    st = sp.FindObject("stats")
    # print(type(st))
    st.SetY1NDC(Yinit - i*0.08)
    st.SetY2NDC(0.1)
lg.Draw()
c.Draw()
ROOT.gPad.Modified()
ROOT.gPad.Update()

Warning in <TROOT::Append>: Replacing existing TH1: Ch. 14 No correction (Potential memory leak).


In [63]:
if ROOT.gROOT.FindObject('cv_4plots'):
    ROOT.gROOT.FindObject('cv_4plots').Close()

canvas4 = ROOT.TCanvas('cv_4plots', 'cv_4plots', 1500, 900)
canvas4.Divide(2,2)

legends = [ROOT.TLegend(0.48, 0.61, 0.75, 0.83) for _ in range(4)]

canvas4.cd(1)
hists[0].Draw('hist')
hists[3].Draw('sames hist')
legends[0].AddEntry(hists[0], "Baseline subtracted", 'l')
legends[0].AddEntry(hists[3], "Not corrected", 'l')
legends[0].SetHeader("\'Summed\' channel spectra")
legends[0].Draw()
canvas4.cd(1).SetLogy()

canvas4.cd(2)
hists[1].SetXTitle("ADC channel number")
hists[1].SetYTitle("Counts")
hists[1].Draw('hist')
hists[4].Draw('sames hist')
legends[1].AddEntry(hists[1], "Baseline subtracted", 'l')
legends[1].AddEntry(hists[4], "Not corrected", 'l')
legends[1].SetHeader("Ch. 2 spectra")
legends[1].Draw()
canvas4.cd(2).SetLogy()

canvas4.cd(3)
hists[2].SetXTitle("ADC channel number")
hists[2].SetYTitle("Counts")
hists[2].Draw('hist')
hists[5].Draw('sames hist')
legends[2].AddEntry(hists[2], "Baseline subtracted", 'l')
legends[2].AddEntry(hists[5], "Not corrected", 'l')
legends[2].SetHeader("Ch. 14 spectra")
legends[2].Draw()
canvas4.cd(3).SetLogy()

canvas4.cd(4)
h0_zoom = hists[0].Clone("hists_0_zoomed")
h0_zoom.GetXaxis().SetRangeUser(0,BITS12)
h0_zoom.Draw('hist')
hists[1].Draw('sames hist')
hists[2].Draw('sames hist')
legends[3].AddEntry(h0_zoom, "\'Summed\'", 'l')
legends[3].AddEntry(hists[1], "Ch. 2", 'l')
legends[3].AddEntry(hists[2], "Ch. 14", 'l')
legends[3].SetHeader("Corrected channels")
legends[3].Draw()
canvas4.cd(4).SetLogy()

canvas4.Draw()

In [5]:
print("ADC channels range:\n"
      f"{"":->50}\n"
      "Channel\t\tBL-subtracted\tNo correction\n"
      f"{"":->50}\n"
      f"Summed\t\t{acq_subtracted['s'].min()/2} - {acq_subtracted['s'].max()/2}\t{acq_notSubtracted['s'].min()/2} - {acq_notSubtracted['s'].max()/2}\n"
      f"Ch. 2\t\t{acq_subtracted[2].min()} - {acq_subtracted[2].max()}\t{acq_notSubtracted[2].min()} - {acq_notSubtracted[2].max()}\n"
      f"Ch. 14\t\t{acq_subtracted[14].min()} - {acq_subtracted[14].max()}\t{acq_notSubtracted[14].min()} - {acq_notSubtracted[14].max()}\n")

print("\n\n"
      "Number of values above single channel range in \'Summed\' channel:\n"
      f"{"":->30}\n"
      "BL-subtracted\tNo correction\n"
      f"{"":->30}\n"
      f"{len(acq_subtracted['s'][acq_subtracted['s'][:]/2>BITS12]):^13}\t{len(acq_notSubtracted['s'][acq_notSubtracted['s'][:]/2>BITS12]):^13}")

ADC channels range:
--------------------------------------------------
Channel		BL-subtracted	No correction
--------------------------------------------------
Summed		0.0 - 3965.0	935.5 - 4880.5
Ch. 2		0.0 - 3951.0	51.0 - 4095.0
Ch. 14		0.0 - 3979.0	0.0 - 4095.0



Number of values above single channel range in 'Summed' channel:
------------------------------
BL-subtracted	No correction
------------------------------
      0      	     22      


The baseline subtraction is performed directly at the conversion script. No averaging over the number of channels is carried out by the conversion script, but the above spectra and data correspond to values averaged over the number of active channels (2).

The values of the baselines of all the channels are between 98 and 150.

The range of the summed channel does not exceed the maximum single-channel value (4096) after baseline correction.

**<span style="color:red">Questions:</span>**
- Is this baseline correction enough for pedestal subtraction?
- What about the large peak at the lower range of the spectrum? Is it still originating from noise at this stage?

# Calibration of the summed spectrum with the backgrond peaks

In [23]:
arr = np.array([1,2,3,4,5])
arr[[True, True, True, False, False]]

array([1, 2, 3])

In [4]:
# def gauspeak(n_init):
#     return f"[{n_init}]*TMath::Gaus(x, [{n_init+1}], [{n_init+2}]"

# formula = "[0]*TMath::Exp(-x/[1]) + [2]*TMath::Gaus(x, [3], [4])"

# c2 = ROOT.TCanvas("c2", "c2", 800, 600)
# # bg_exp = ROOT.TF1("ExpDecayBG", "[p0]*TMath::Exp(x*[p1]/[p2])", 0, BITS_12)
# # gaussian1 = ROOT.TF1("K40peak", "TMath::Gaus(x, [mean], [sigma])", 800, 1100)
# # gaussian2 = ROOT.TF1("Tl208peak", "TMath::Gaus(x, [mean], [sigma])", 1300, 1600)

# bg = ROOT.TF1("ExpDecayBG", formula, 700, 1200, 5)
# bg.SetParNames("exp_fact", "exp_denom", "norm", "mean", "sigma")
# bg.SetParameters(180, 200, 4, 904, 4)

# fitResult = ch_spectra[2].Fit(bg, "L S", "", 700, 1200)
# ch_spectra[2].GetXaxis().SetRangeUser(0, BITS_12)
# ch_spectra[2].Draw("SAME hist")
# c2.SetLogy()
# c2.Draw()

In [5]:
# def gauspeak(n_init):
#     return f"[{n_init}]*TMath::Gaus(x, [{n_init+1}], [{n_init+2}])"

# def bg_exp(n_init):
#     return f"[{n_init}]*TMath::Exp(-x/[{n_init+1}])"
# norm = bg.GetParameter(2)
# mean = bg.GetParameter(3)
# sigma = bg.GetParameter(4)
# const = bg.GetParameter(0)
# expdenom = bg.GetParameter(1)
# c3 = ROOT.TCanvas("c3", "c3", 800, 600)
# peak = ROOT.TF1("K40_peak", gauspeak(0), 400, 1500, 3)
# peak.SetParameters(norm, mean, sigma)
# peak.SetLineColor(ROOT.kMagenta)
# onlybg = ROOT.TF1("onlybg", "[0]*TMath::Exp(-x/[1])", 100, 2000, 2)
# onlybg.SetParameters(const, expdenom)
# onlybg.SetLineColor(ROOT.kBlue)
# onlybg.SetLineStyle(2)
# ch_spectra[2].Draw("SAME hist")
# peak.Draw("same")
# onlybg.Draw("same")
# c3.SetLogy()
# c3.Draw()

NameError: name 'bg' is not defined

In [6]:
def gauspeak(n_init):
    return f"[{n_init}]*TMath::Gaus(x, [{n_init+1}], [{n_init+2}])"

def bg_exp(n_init):
    return f"[{n_init}]*TMath::Exp(-x/[{n_init+1}])"

def peak_and_bg():
    return f"{bg_exp(0)}+{gauspeak(2)}"

def fitpeak(hsit, name, xl, xr, norm, mean, sigma=70, const=200, denom=200, showFit=False, keep_prev_fncs=True):
    fit_fn = ROOT.TF1(name, peak_and_bg(), xl, xr, 5)
    fit_fn.SetParNames("Const", "Denom", "Norm", "Mean", "Sigma")
    fit_fn.SetParameters(const, denom, norm, mean, sigma)
    options = "L S" if showFit else "0 S"
    options += " +" if keep_prev_fncs else ""
    fit_result = hist.Fit(fit_fn, options, "", xl, xr)
    return fit_fn, fit_result


# fit_fn_ch14, fit_ch14 = fitpeak(ch_spectra[3], name="K40_peak", xl=650, xr=1200, norm=4, mean=840)

# c4 = ROOT.TCanvas("c4", "c4", 800, 600)
# ch_spectra[3].GetXaxis().SetRangeUser(0, BITS_12)
# # ch_spectra[3].GetListOfFunctions().Add("K40_peak")
# ch_spectra[3].Draw("hist")
# c4.SetLogy()
# c4.Draw()

In [7]:
k40peak_init = 840
xrange_k40 = (650, 1200)
norms_init = [1,3,4,3]
sig_init = 70

fit_fns = []
k40peaks_means = []
k40peaks_sigmas = []

for hist, nrm, ch  in zip(ch_spectra, norms_init, legend):
    fit_fn, fit_res = fitpeak(hist, name="K40_peak", xl=xrange_k40[0], xr=xrange_k40[1], norm=nrm, mean=k40peak_init, showFit=True)
    fit_fns.append(fit_fn)
    k40peaks_means.append(fit_fns[-1].GetParameter(3))
    k40peaks_sigmas.append(fit_fns[-1].GetParameter(4))

c5 = ROOT.TCanvas("c4", "c4", 800, 600)
# ch_spectra[3].GetListOfFunctions().Add("K40_peak")

for sp in ch_spectra:
    sp.Draw("hist,sames")
c5.SetLogy()
c5.Draw()

****************************************
Minimizer is Minuit2 / Migrad
MinFCN                    =     0.866612
Chi2                      =      1.73322
NDf                       =           64
Edm                       =  1.48724e-06
NCalls                    =         1442
Const                     =       22.546   +/-   26.3142     
Denom                     =      234.686   +/-   86.4285     
Norm                      =     0.634481   +/-   0.336032    
Mean                      =      894.211   +/-   40.4785     
Sigma                     =     -68.8513   +/-   58.3939     
****************************************
Minimizer is Minuit2 / Migrad
MinFCN                    =     0.573121
Chi2                      =      1.14624
NDf                       =           64
Edm                       =   4.3817e-07
NCalls                    =          204
Const                     =      120.222   +/-   75.8598     
Denom                     =      209.018   +/-   33.8978     
Norm          

In [8]:
tl208peak_init = 1380
xrange_tl208 = (1100, 1800)
norms_init = [0.1, 0.4, 0.4, 0.5]
sig_init = 70

fit_fns = []
tl208peaks_means = []
tl208peaks_sigmas = []

for hist, nrm, ch  in zip(ch_spectra, norms_init, legend):
    fit_fn, fit_res = fitpeak(hist, name="Tl208peak", xl=xrange_tl208[0], xr=xrange_tl208[1], norm=nrm, mean=tl208peak_init, showFit=True)
    fit_fns.append(fit_fn)
    tl208peaks_means.append(fit_fns[-1].GetParameter(3))
    tl208peaks_sigmas.append(fit_fns[-1].GetParameter(4))

c5 = ROOT.TCanvas("c5", "c5", 800, 600)
# ch_spectra[3].GetListOfFunctions().Add("K40_peak")

for sp in ch_spectra:
    sp.Draw("hist,sames")
c5.SetLogy()
c5.Draw()

****************************************
Minimizer is Minuit2 / Migrad
MinFCN                    =     0.948685
Chi2                      =      1.89737
NDf                       =           83
Edm                       =  8.42893e-06
NCalls                    =          628
Const                     =       396.93   +/-   3632.12     
Denom                     =      148.287   +/-   179.97      
Norm                      =    0.0896652   +/-   0.0935118   
Mean                      =      1443.17   +/-   191.88      
Sigma                     =      104.392   +/-   145.365     
****************************************
Minimizer is Minuit2 / Migrad
MinFCN                    =     0.560084
Chi2                      =      1.12017
NDf                       =           83
Edm                       =  4.59517e-06
NCalls                    =          634
Const                     =      4562.96   +/-   35201.8     
Denom                     =      123.626   +/-   106.668     
Norm          

In [9]:
c5 = ROOT.TCanvas("c5", "c5", 800, 600)
# ch_spectra[3].GetListOfFunctions().Add("K40_peak")
t = []
for sp in ch_spectra:
    sp.Draw("hist,sames")
    tl208_fn = sp.GetFunction("Tl208peak")
    text_x = tl208_fn.GetParameter(3)
    text_y = tl208_fn.Eval(text_x)
    t.append(ROOT.TText(text_x, text_y,f"{text_x:.2f}"))
    t[-1].SetTextSize(10)
    t[-1].SetTextAngle(90)
    t[-1].Draw()
    # sp.GetFunction("Tl208peak").Delete()
for sp in ch_spectra:
    k40_fn = sp.GetFunction("K40_peak")
    text_x = k40_fn.GetParameter(3)
    text_y = k40_fn.Eval(text_x)
    t.append(ROOT.TText(text_x, text_y,f"{text_x:.2f}"))
    t[-1].SetTextSize(10)
    t[-1].SetTextAngle(90)
    t[-1].Draw()

IamLegend.Draw()
c5.SetLogy()
c5.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c5


In [ ]:
#def get_peak_from_fit(fit_fn, 

In [10]:
K40_MEV = 1.460
TL208_MEV = 2.614

raw_data = frame_lst[3][SIPHRA_Ch_list[3]].to_numpy()

calib_m = (TL208_MEV - K40_MEV) / (tl208peaks_means[3] - k40peaks_means[3])
calib_b = K40_MEV - calib_m*k40peaks_means[3]
print(f"\nCalibration: E = {calib_m:.4f} * Ch + {calib_b:.4f}\n")

calib_data = (raw_data * calib_m + calib_b)
print(f"Max. Energy: {BITS_12*calib_m+calib_b:.2f} MeV")

hist_calib = ROOT.TH1F("Ch14_BG_calib", "Spectrum calibrated w.r.t. K40 & Tl208", N_BINS, min(calib_data), max(calib_data))

hist_calib.Fill(calib_data)
hist_calib.Scale(1/times_SiPM14_BG[3])
hist_calib.SetLineColor(colors[3]-3)

c = ROOT.TCanvas("c", "c", 800, 600)
hist_calib.Draw("hist")
c.SetLogy()
c.Draw()



Calibration: E = 0.0023 * Ch + -0.5498

Max. Energy: 9.02 MeV


In [28]:
source_raw_dat = dfs_SiPM14_Cs137[3][SIPHRA_Ch_list[3]].to_numpy()
source_calib_data = source_raw_dat*calib_m + calib_b
source_hist_calib = ROOT.TH1F("Ch14_Cs137_calib", "", N_BINS, min(source_calib_data), max(source_calib_data))
source_hist_calib.Fill(source_calib_data)
source_hist_calib.Scale(1/times_SiPM14_Cs137[3])

fitpeak(source_hist_calib, name="Cs137peak", xl=0.2, xr=1.2, norm=170, mean=0.66, sigma=0.5, const=200, denom=0.1, showFit=True)

hist_calib.GetYaxis().SetRangeUser(0,2e2)

c = ROOT.TCanvas("c", "c", 800, 600)
source_hist_calib.Draw("hist same")
source_hist_calib.GetXaxis().SetTitle("Energy (MeV)")
source_hist_calib.GetYaxis().SetTitle(r"Count Rate (s^{-1})")
hist_calib.Draw("hist same")
hist_calib.SetFillColorAlpha(colors[3]-3, 0.5)
source_hist_calib.SetFillColor(ROOT.kBlue)

c.SetLogy()
c.Draw()


****************************************
         Invalid FitResult  (status = 2 )
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =            0
NDf                       =            0
Edm                       =  1.06209e-16
NCalls                    =           89
Const                     =          200   +/-   0           
Denom                     =          0.1   +/-   0           
Norm                      =     -103.981   +/-   0           
Mean                      =         0.66   +/-   0           
Sigma                     =          0.5   +/-   0           


Warning in <TROOT::Append>: Replacing existing TH1: Ch14_Cs137_calib (Potential memory leak).
Warning in <Fit>: Abnormal termination of minimization.
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [ ]:
fit_fn, fit_res = fitpeak(hist, name="Tl208peak", xl=xrange_tl208[0], xr=xrange_tl208[1], norm=nrm, mean=tl208peak_init, showFit=True)
    fit_fns.append(fit_fn)
    tl208peaks_means.append(fit_fns[-1].GetParameter(3))
    tl208peaks_sigmas.append(fit_fns[-1].GetParameter(4))